<a href="https://colab.research.google.com/github/mayanku1111/SFprojects/blob/main/Magicapturee7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
!pip install torch==1.12.0 torchvision==0.13.0 torchaudio==0.12.0

In [13]:
%cd /content/drive/MyDrive/MagiCapture/insightface_/detection/retinaface

/content/drive/MyDrive/MagiCapture/insightface_/detection/retinaface


In [12]:
!pip install -r /content/drive/MyDrive/MagiCapture/requirements.txt

  Using cached mxnet-1.6.0-py2.py3-none-any.whl.metadata (3.4 kB)
  Using cached mxnet_cu112-1.9.1-py3-none-manylinux2014_x86_64.whl.metadata (3.5 kB)
Using cached mxnet-1.6.0-py2.py3-none-any.whl (68.7 MB)
Using cached mxnet_cu112-1.9.1-py3-none-manylinux2014_x86_64.whl (499.4 MB)


In [11]:
!pip install jax==0.4.23 jaxlib==0.4.23 diffusers==0.14.0

In [10]:
import os
os.environ['MODEL_NAME'] = "runwayml/stable-diffusion-v1-5"
os.environ['INSTANCE_DIR'] = "data/instance_data"
os.environ['CLASS_DIR'] = "data/class_data"
os.environ['STYLE_DIR'] = "data/style_data"
os.environ['OUTPUT_DIR'] = "output"

In [9]:
import numpy as np

In [4]:
!pip install cython

In [5]:
%%writefile bbox_overlaps.pyx

import numpy as np
cimport numpy as np

ctypedef np.float64_t DTYPE_t

def bbox_overlaps_cython(
        np.ndarray[DTYPE_t, ndim=2] boxes,
        np.ndarray[DTYPE_t, ndim=2] query_boxes):
    """
    Parameters
    ----------
    boxes: (N, 4) ndarray of float
    query_boxes: (K, 4) ndarray of float
    Returns
    -------
    overlaps: (N, K) ndarray of overlap between boxes and query_boxes
    """
    cdef unsigned int N = boxes.shape[0]
    cdef unsigned int K = query_boxes.shape[0]
    cdef np.ndarray[DTYPE_t, ndim=2] overlaps = np.zeros((N, K), dtype=np.float64)
    cdef DTYPE_t iw, ih, box_area
    cdef DTYPE_t ua
    cdef unsigned int k, n

    for k in range(K):
        box_area = (
            (query_boxes[k, 2] - query_boxes[k, 0] + 1) *
            (query_boxes[k, 3] - query_boxes[k, 1] + 1)
        )
        for n in range(N):
            iw = (
                min(boxes[n, 2], query_boxes[k, 2]) -
                max(boxes[n, 0], query_boxes[k, 0]) + 1
            )
            if iw > 0:
                ih = (
                    min(boxes[n, 3], query_boxes[k, 3]) -
                    max(boxes[n, 1], query_boxes[k, 1]) + 1
                )
                if ih > 0:
                    ua = (
                        (boxes[n, 2] - boxes[n, 0] + 1) *
                        (boxes[n, 3] - boxes[n, 1] + 1) +
                        box_area - iw * ih
                    )
                    overlaps[n, k] = iw * ih / ua
    return overlaps

Writing bbox_overlaps.pyx


In [7]:
# from rcnn.config import config

In [13]:
%cd /content/drive/MyDrive/MagiCapture

/content/drive/MyDrive/MagiCapture


In [9]:
%cd /content/drive/MyDrive/MagiCapture/insightface_/detection/retinaface/rcnn/cython

/content/drive/MyDrive/MagiCapture/insightface_/detection/retinaface/rcnn/cython


In [11]:
!pip install cupy
!python setup.py build_ext --inplace

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 47.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for cupy
  Running setup.py clean for cupy
Failed to build cupy
ERROR: ERROR: Failed to build installable wheels for some pyproject.toml based projects (cupy)
running build_ext
building 'gpu_nms' extension
error: unknown file type '.cu' (from 'nms_kernel.cu')


In [35]:
# from setuptools import setup
# from Cython.Build import cythonize
# import numpy as np

# setup(
#     ext_modules = cythonize("bbox.pyx"),
#     include_dirs=[np.get_include()]
# )

In [17]:
!CUDA_VISIBLE_DEVICES=1 python /content/drive/MyDrive/MagiCapture/lora_diffusion_/magicapture_train.py \
  --pretrained_model_name_or_path=/content/drive/MyDrive/MagiCapture/arcface_torch/models/ms1mv3_arcface_r100_fp16 \
  --instance_data_dir=/content/drive/MyDrive/MagiCapture/data/instance_data \
  --style_data_dir=/content/drive/MyDrive/MagiCapture/data/style_data \
  --output_dir=/content/drive/MyDrive/MagiCapture \
  --resolution=512 \
  --with_prior_preservation=False \
  --train_text_encoder \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 --gradient_checkpointing \
  --scale_lr \
  --initializer_tokens="person" \
  --learning_rate_unet=1e-4 \
  --learning_rate_text=1e-5 \
  --learning_rate_ti=5e-4 \
  --color_jitter \
  --lr_scheduler="linear" \
  --lr_warmup_steps=0 \
  --placeholder_tokens="<sks>" \
  --placeholder_tokens_style="<style1>" \
  --save_steps=8000 \
  --max_train_steps_ti=1200 \
  --max_train_steps_tuning=1500 \
  --perform_inversion=True \
  --clip_ti_decay \
  --weight_decay_ti=0.000 \
  --weight_decay_lora=0.001 \
  --continue_inversion \
  --continue_inversion_lr=1e-4 \
  --device="cuda:0" \
  --lora_rank=4 \
  --log_wandb=False \
  --lambda_arc=0.25 \
  --lambda_style=2

2024-07-27 19:57:45.326162: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-27 19:57:45.326213: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-27 19:57:45.327591: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-27 19:57:46.434046: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/content/drive/MyDrive/MagiCapture
['/content/drive/MyDrive/MagiCapture/lora_diffusion_', '/env/python', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '/usr/local/lib/python3.10/dist-packages', '/usr/lib

In [6]:
!ls

arcface_torch	    environment.yaml	      inference.sh     __pycache__
build_eval_pack.py  face_align.py	      insightface_     README.md
CodeFormer	    face_parsing_PyTorch      LICENSE	       requirements.txt
data		    inference_magicapture.py  lora_diffusion_  train_magicapture.sh


In [7]:
import sys
sys.path.append('/content/drive/MyDrive/MagiCapture')

In [3]:
!pip install cython


In [15]:
%%writefile setup.py

from setuptools import setup
from Cython.Build import cythonize
from distutils.extension import Extension
import numpy as np

extensions = [
    Extension(
        "bbox",
        ["bbox.pyx"],
        extra_compile_args=['-Wno-cpp', '-Wno-unused-function']
    ),
    Extension(
        "anchors",
        ["anchors.pyx"],
        extra_compile_args=['-Wno-cpp', '-Wno-unused-function']
    ),
    Extension(
        "cpu_nms",
        ["cpu_nms.pyx"],
        extra_compile_args=['-Wno-cpp', '-Wno-unused-function']
    ),
    Extension(
        "gpu_nms",
        ["gpu_nms.pyx", "nms_kernel.cu"],
        library_dirs=['/usr/local/cuda/lib64'],
        libraries=['cudart'],
        language='c++',
        runtime_library_dirs=['/usr/local/cuda/lib64'],
        extra_compile_args={
            'gcc': ['-Wno-unused-function'],
            'nvcc': ['-arch=sm_75']  # This is correct for T4 GPU
        }
    )
]

setup(
    ext_modules=cythonize(extensions),
    include_dirs=[np.get_include(), '/usr/local/cuda/include']
)

Overwriting setup.py
